## test MongoDB function

In [1]:
from src.dataread import read_file

In [2]:
!ls

LICENSE            app                yelp-api.ipynb     yelp_api_key.yaml
MongoDB.ipynb      data               yelp-ver1.ipynb    yelp_dataset
README.md          request_example.py yelp.ipynb         yelp_helpers.py
__pycache__        src                yelp.py


In [2]:
businesses = read_file('./yelp_dataset/yelp_academic_dataset_business.json')

In [3]:
businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


### load json files and store in MongoDB

In [4]:
from pymongo import MongoClient

In [5]:
# Connect to the hosted MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

In [9]:
def read_json(file_name):
    '''
    Read the json file into a list
    INPUT: string
    OUTPUT: list
    '''
    with open(file_name, 'r') as json_file:
        list_json = json_file.readlines()
    return list_json

In [10]:
def json_to_mongo(file_name, db_name, collection_name):
    '''
    Read json file and store to mongodb
    INPUT: string, string, string
    OUTPUT: None
    '''
    list_json = read_json(file_name)
    client = MongoClient()
    db = client[db_name]
    collection = db[collection_name]
    for json_item in list_json:
        record = json.loads(json_item)
        collection.insert(record)

In [11]:
file_name = './yelp_dataset/yelp_academic_dataset_business.json'
db_name = 'yelp'
collection_name = 'business'

In [12]:
json_to_mongo(file_name, db_name, collection_name)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


In [13]:
file_name = './yelp_dataset/yelp_academic_dataset_review.json'

In [14]:
json_to_mongo(file_name, db_name, collection_name='review')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


In [15]:
file_name = './yelp_dataset/yelp_academic_dataset_user.json'

In [17]:
json_to_mongo(file_name, db_name, collection_name='user')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


accidently stored some user data into review collections, remove them using the following

`db.review.remove({_id:{$gte:ObjectId("5b8865c11461653d76a27892")}})`

### read MongoDB to pandas

In [18]:
db = client.yelp

In [21]:
db.list_collection_names()

['review', 'user', 'business']

In [27]:
import pandas as pd

In [42]:
# Make a query to the yelp DB and business Collection
cursor = db.business.find({'city': 'Phoenix', 'categories':{'$regex': 'Restaurant'}})

In [43]:
# Expand the cursor and construct the DataFrame
business =  pd.DataFrame(list(cursor))

In [44]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3839 entries, 0 to 3838
Data columns (total 16 columns):
_id             3839 non-null object
address         3839 non-null object
attributes      3792 non-null object
business_id     3839 non-null object
categories      3839 non-null object
city            3839 non-null object
hours           2879 non-null object
is_open         3839 non-null int64
latitude        3839 non-null float64
longitude       3839 non-null float64
name            3839 non-null object
neighborhood    3839 non-null object
postal_code     3839 non-null object
review_count    3839 non-null int64
stars           3839 non-null float64
state           3839 non-null object
dtypes: float64(3), int64(2), object(11)
memory usage: 480.0+ KB


In [46]:
columns = ['business_id', 'categories', 'attributes']

In [47]:
biz = business[columns]

In [67]:
review_cursor = db.review.find({'user_id': '--2HUmLkcNHZp0xw6AMBPg'})
review_user = pd.DataFrame(list(review_cursor))
review_user

,_id,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,5b8863461461653d76932d4b,Xfod0UWr_9B-TOM6qxVIwQ,0,2017-06-17,0,chp1hXWLG4HHiu3RiZFBZQ,5,Great price point to plate ratio. Amazing sus...,0,--2HUmLkcNHZp0xw6AMBPg
1,5b8863461461653d76932d4c,NWdSHWpE2Ms93mevNK2A6g,0,2017-07-19,0,6pSl_HGoact6MC-7DiGB-w,5,My husband and I were recommended by a friend....,0,--2HUmLkcNHZp0xw6AMBPg
2,5b8863461461653d76932d4d,3-6biVwm7VwPZ-k67jml-Q,0,2017-06-17,0,1E5teQq-jatUnX3_1H7VNA,2,"Food was really good, service was TERRIBLE. I...",4,--2HUmLkcNHZp0xw6AMBPg
3,5b8863461461653d76932d4e,XDRWtl3p4EfCxAMNYEOqeg,0,2017-06-17,0,pIq--JSuE1kduMe40gzGgw,5,Service here is incredible. The nicest staff....,0,--2HUmLkcNHZp0xw6AMBPg
4,5b8863461461653d76932d4f,c5x6HWB8MTZvlTgabihaSw,1,2017-07-04,0,NYbX5FZv_ci6-gbnlJ-DVQ,5,Absolutely obsessed with this place. I am a h...,0,--2HUmLkcNHZp0xw6AMBPg
5,5b8863461461653d76932d50,pEW4vERzojY2zMk1hvp5kQ,0,2018-05-22,0,z-e23mhL8YQblsm36y1JAg,5,Had the best experience in your store on Sunda...,0,--2HUmLkcNHZp0xw6AMBPg
6,5b8863461461653d76932d51,7m1Oa1VYV98UUuo_6i0EZg,0,2018-06-07,0,nQg-GWFAPlzU0ouK5-0bWg,5,First time coming by! Saw the reviews on line...,0,--2HUmLkcNHZp0xw6AMBPg
7,5b8863461461653d76932d52,Iq7NqQD-sESu3vr9iEGuTA,0,2017-06-17,0,528YVt_R319BrWrkPv7cLw,5,Such good breakfast. Love the pancakes - so m...,1,--2HUmLkcNHZp0xw6AMBPg
8,5b8863461461653d76932d53,6XpqswYWoSOx0a9MkosVRA,0,2017-06-17,0,ZCAbFH-gnUcz6c8ANEp_aw,4,This is a hard one. The tacos and food are RE...,0,--2HUmLkcNHZp0xw6AMBPg
9,5b8863461461653d76932d54,K-vBVV_RZX-BeyqOBWjsDg,0,2017-06-17,0,Rs_cxx89-YEJNUlvOSJsug,5,I am obsessed with their zipperitas. It's the...,0,--2HUmLkcNHZp0xw6AMBPg


In [49]:
col_review = ['business_id', 'stars', 'user_id']

In [50]:
review = review_user[col_review]

In [88]:
row_list = []
for id in review['business_id']:
    business_cursor = db.business.find({'business_id': id})
    row_list.append(list(business_cursor))
    

In [91]:
row_list[0][0]

{'_id': ObjectId('5b8856ec1461653d76452fc1'),
 'business_id': 'Xfod0UWr_9B-TOM6qxVIwQ',
 'name': 'Asian Island',
 'neighborhood': '',
 'address': '14870 N Northsight Blvd, Ste 103',
 'city': 'Scottsdale',
 'state': 'AZ',
 'postal_code': '85260',
 'latitude': 33.6216054,
 'longitude': -111.8990655,
 'stars': 4.0,
 'review_count': 166,
 'is_open': 1,
 'attributes': {'Alcohol': 'none',
  'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}",
  'BikeParking': 'True',
  'BusinessAcceptsCreditCards': 'True',
  'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  'Caters': 'True',
  'GoodForKids': 'True',
  'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'breakfast': False, 'brunch': False}",
  'HasTV': 'True',
  'NoiseLevel': 'average',
  'OutdoorSeating': 'True',
  'RestaurantsAtti

In [109]:
business = pd.DataFrame()

In [110]:
for i in range(len(row_list)):
    business = pd.concat([business, pd.DataFrame(row_list[i])], ignore_index=True)

In [111]:
business

,_id,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,5b8856ec1461653d76452fc1,"14870 N Northsight Blvd, Ste 103","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",Xfod0UWr_9B-TOM6qxVIwQ,"Chinese, Hawaiian, Restaurants, Asian Fusion, ...",Scottsdale,"{'Monday': '10:30-21:0', 'Tuesday': '10:30-21:...",1,33.621605,-111.899066,Asian Island,,85260,166,4.0,AZ
1,5b8856e91461653d76451c71,"6920 E Shea Blvd, Ste 103","{'AcceptsInsurance': 'True', 'BusinessAcceptsC...",NWdSHWpE2Ms93mevNK2A6g,"Health & Medical, Doctors, Medical Centers, Sp...",Scottsdale,"{'Monday': '14:0-18:0', 'Tuesday': '8:30-12:0'...",1,33.582581,-111.931602,Connected Medical Center,,85254,21,5.0,AZ
2,5b8856d21461653d7644752d,8880 E Via Linda,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",3-6biVwm7VwPZ-k67jml-Q,"Bakeries, Food, Cheese Shops, Specialty Food, ...",Scottsdale,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,33.569529,-111.889629,Andreoli Italian Grocer,,85258,368,4.0,AZ
3,5b8857101461653d764631e1,7386 E Shea Blvd,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",XDRWtl3p4EfCxAMNYEOqeg,"Chicken Shop, American (Traditional), Fast Foo...",Scottsdale,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,33.582229,-111.922303,Raising Cane's,,85260,34,3.5,AZ
4,5b8857071461653d7645f444,7101 E Thunderbird Rd,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",c5x6HWB8MTZvlTgabihaSw,"Restaurants, Italian",Scottsdale,"{'Monday': '16:30-20:30', 'Tuesday': '16:30-20...",1,33.610999,-111.927610,Arrivederci,,85254,233,4.0,AZ
5,5b8856d61461653d7644954d,1845 E Baseline Rd,"{'BikeParking': 'True', 'BusinessAcceptsCredit...",pEW4vERzojY2zMk1hvp5kQ,"Grocery, Shopping, Food, Drugstores",Gilbert,None,1,33.378923,-111.791606,Fry's,,85233,43,3.5,AZ
6,5b8857001461653d7645c2d8,"4001 E Bell Rd, Ste 102","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",7m1Oa1VYV98UUuo_6i0EZg,"American (Traditional), Burgers, Mexican, Rest...",Phoenix,"{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'...",1,33.639913,-111.995703,Paradise Valley Burger Company,,85032,1131,4.5,AZ
7,5b88571a1461653d76467467,"8390 E Via De Ventura, Ste F-108","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",Iq7NqQD-sESu3vr9iEGuTA,"Waffles, Breakfast & Brunch, Coffee & Tea, Res...",Scottsdale,"{'Monday': '6:30-14:30', 'Tuesday': '6:30-14:3...",1,33.554825,-111.899636,Butters Pancakes & Café,,85258,1167,4.5,AZ
8,5b8857161461653d7646594a,7001 N Scottsdale Rd,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",6XpqswYWoSOx0a9MkosVRA,"Restaurants, Mexican",Scottsdale,"{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",0,33.539308,-111.922631,Tia Shorty's Authentic Mexican Food,,85253,142,4.0,AZ
9,5b8856f31461653d7645683c,8380 E Via De Ventura,"{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",K-vBVV_RZX-BeyqOBWjsDg,"Restaurants, Nightlife, American (New), Americ...",Scottsdale,"{'Monday': '11:0-0:30', 'Tuesday': '11:0-0:30'...",1,33.554744,-111.900222,Zipps Sports Grill,,85258,183,3.5,AZ


In [52]:
user = pd.merge(biz, review, how='inner', on='business_id')

In [53]:
user

,business_id,categories,attributes,stars,user_id
0,cHdJXLlKNWixBXpDwEGb_A,"Szechuan, Restaurants, Mexican, Asian Fusion, ...","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",5,--2HUmLkcNHZp0xw6AMBPg
1,LR_99E7tVUfxqemvcGkDzw,"Mediterranean, Restaurants, Breakfast & Brunch...","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",3,--2HUmLkcNHZp0xw6AMBPg
2,E4JyAzB5_2quptwtemyhYA,"Breakfast & Brunch, Burgers, Seafood, Restaura...","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",5,--2HUmLkcNHZp0xw6AMBPg
3,7m1Oa1VYV98UUuo_6i0EZg,"American (Traditional), Burgers, Mexican, Rest...","{'Alcohol': 'none', 'Ambience': '{'romantic': ...",5,--2HUmLkcNHZp0xw6AMBPg
4,u9R0_pvEkl7QGPJYH3j1sg,"Italian, Seafood, Restaurants, Pizza","{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",5,--2HUmLkcNHZp0xw6AMBPg
5,yX0pOWG_Be9TjGnm0aihVA,"Breakfast & Brunch, Food, Restaurants, America...","{'Alcohol': 'full_bar', 'Ambience': '{'romanti...",5,--2HUmLkcNHZp0xw6AMBPg


In [60]:
for i in range(len(user)):
    print(user.categories[i])

Szechuan, Restaurants, Mexican, Asian Fusion, Chinese
Mediterranean, Restaurants, Breakfast & Brunch, American (Traditional)
Breakfast & Brunch, Burgers, Seafood, Restaurants, Cafes
American (Traditional), Burgers, Mexican, Restaurants
Italian, Seafood, Restaurants, Pizza
Breakfast & Brunch, Food, Restaurants, American (New), Burgers, Comfort Food, Beer, Wine & Spirits, Vegetarian


In [62]:
cat_words = []
for i in range(len(user)):
    for word in user.categories[i].split(', '):
        cat_words.append(word)

In [64]:
from collections import Counter

In [66]:
Counter(cat_words).most_common()

[('Restaurants', 6),
 ('Breakfast & Brunch', 3),
 ('Burgers', 3),
 ('Mexican', 2),
 ('American (Traditional)', 2),
 ('Seafood', 2),
 ('Szechuan', 1),
 ('Asian Fusion', 1),
 ('Chinese', 1),
 ('Mediterranean', 1),
 ('Cafes', 1),
 ('Italian', 1),
 ('Pizza', 1),
 ('Food', 1),
 ('American (New)', 1),
 ('Comfort Food', 1),
 ('Beer', 1),
 ('Wine & Spirits', 1),
 ('Vegetarian', 1)]

In [ ]:
db.business